In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import warnings
warnings.filterwarnings("ignore")
import numpy as np

## 檢查所有有替代料號的PCODE和價格

In [55]:
codes = pd.read_clipboard()

In [56]:
results = codes.groupby(["Main PCODE"])["Old PCODEs"].apply(list).reset_index()

In [57]:
pcode_dict = dict(zip(results["Main PCODE"], results["Old PCODEs"]))

In [58]:
old_dict = dict(zip(codes["Old PCODEs"], codes["Main PCODE"]))

## 讀取需要放在B2B的價格表

- 處理TBR價格

In [18]:
tbr_price_23 = pd.read_clipboard()

In [21]:
tbr_price_23 = tbr_price_23[["料號", "規格", "花紋", "23 New List", "Basic net"]]

In [31]:
tbr_price_23["目前料號"] = tbr_price_23["料號"].map(old_dict)

In [33]:
tbr_price_23["其他料號"] = tbr_price_23["目前料號"].map(pcode_dict)

In [37]:
tbr_price_23 = tbr_price_23.explode("其他料號").reset_index(drop=True)

In [41]:
tbr_price_23["折扣價（含稅）"] = round(tbr_price_23["Basic net"] * 1.05, 0)

In [43]:
tbr_price_23 = tbr_price_23[["其他料號", "規格", "花紋", "23 New List", "折扣價（含稅）"]]

- 處理LSR2

In [62]:
lsr_price_23 = pd.read_clipboard()

In [64]:
lsr_price_23.columns = lsr_price_23.columns.str.strip()

In [63]:
lsr_price_23["折扣價（含稅）"] = lsr_price_23["New price"]

In [65]:
lsr_price_23["目前料號"] = lsr_price_23["料號"].map(old_dict)

In [67]:
lsr_price_23["其他料號"] = lsr_price_23["目前料號"].map(pcode_dict)

In [72]:
lsr_price_23 = lsr_price_23.explode("其他料號").reset_index(drop=True)

In [74]:
lsr_price_23 = lsr_price_23[["其他料號", "規格", "花紋", "New price", "折扣價（含稅）"]]

In [83]:
lsr_price_23.columns = ["其他料號", "規格", "花紋", "23 New List", "折扣價（含稅）"]

## 合併整理完的LSR和TBR

In [85]:
final = pd.concat([tbr_price_23, lsr_price_23])
final.to_clipboard(index=False)